In [144]:
import sys
sys.path.append('..')

from crisismmd_dataset import CrisisMMDataset
import os.path as osp
from torch.utils.data import DataLoader
from paths import modelroot
from tqdm import tqdm

class Args:
    pass


opt = Args()
opt.gpu = 0
opt.batch_size = 1
opt.max_dataset_size = 9999999
# opt.max_dataset_size = 100
opt.crop_size = 224
opt.load_size = 224
opt.debug = False
opt.device = 'cuda'
opt.task = 'task1' 
# opt.task = 'task2_merged'
opt.consistent_only = True
opt.num_classes = 2 if opt.task == 'task1' else 5

dataset = CrisisMMDataset()
dataset.initialize(opt, phase='train', cat='all', task=opt.task, no_transform=True, use_cate=True, consistent_only=opt.consistent_only)
data_loader = DataLoader(
    dataset, batch_size=1, shuffle=False,
)
len(data_loader)
for x in data_loader:
    print(x)
    break

[*] 9601 samples loaded.
{'path_image': ['W:/datasets/CrisisMMD_v2.0/data_image/california_wildfires/10_10_2017/917791291823591425_0.jpg'], 'text': ['RT @Cal_OES: PLS SHARE: Weâ€™re capturing wildfire response, recovery info here: https://t.co/r89LKpjLPj https://t.co/HiA1oQF2Ax'], 'text_tokens': {'input_ids': tensor([[  101,  1030, 10250,  1035,  1051,  2229,  1024, 20228,  2015,  3745,
          1024,  2057,  2050, 30102, 30108,  2890, 11847,  3748, 10273,  3433,
          1010,  7233, 18558,  2182,  1024,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}, 'label_str': ['informative'], 'label': tensor([1]),

In [145]:
from sklearn.svm import SVC
import numpy as np
import torch
import scipy.stats
from sklearn.metrics import accuracy_score

def p2oh(prediction):
    oh = torch.LongTensor(opt.num_classes).zero_()
    oh[prediction] = 1
    return oh.numpy()

def build_xy(extractor, data_loader, data_limit=1000):
    y = []
    x = []
    for idx, data in tqdm(enumerate(data_loader), total=min(data_limit, len(data_loader))):
        if idx > data_limit:
            break
        
        # y.append(p2oh(data['label'].numpy()))
        y.append(data['label'][0].numpy())

        # feat = extractor(data)
        x.append(data['category_vector'][0][0:1].numpy())

    x = np.array(x)
    if x[0].size == 1:
        x = x.reshape(-1, 1)
    print(x)
    y = np.array(y)
    print(y)

    return x, y


def evaluate_svm(x, y):
    clf = SVC(kernel='rbf', C=0.5, gamma='scale')
    clf.fit(x, y)
    pred_y = clf.predict(x)
    true_y = y.squeeze()
    print(true_y, pred_y)
    print(accuracy_score(true_y, pred_y))
    return clf


# Reference: https://scikit-learn.org/0.18/auto_examples/svm/plot_iris.html
def plot(clf, x):
    pass 


def evaluate_corr(x, y):
    return scipy.stats.pearsonr(x, y)


In [146]:
y.shape

(6126,)

In [147]:
sys.path.append('..')
from categorizer import Categorizer
categorizer = Categorizer()
x, y = build_xy(categorizer._get_feat_is_raw, data_loader, 999999)

Loading weights for net_encoder
Loading weights for net_decoder


 52%|█████▏    | 5008/9601 [01:31<01:17, 59.01it/s]E:\anaconda\envs\11777\lib\site-packages\PIL\Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 9601/9601 [02:51<00:00, 55.95it/s]


[[ 0.4991272]
 [ 0.4991272]
 [-2.0034974]
 ...
 [ 0.4991272]
 [ 0.4991272]
 [ 0.4991272]]
[1 1 1 ... 1 1 1]


In [148]:
evaluate_svm(x, y)
# evaluate_svm(nx, y)

[1 1 1 ... 1 1 1] [1 1 0 ... 1 1 1]
0.6629517758566816


SVC(C=0.5)

In [101]:
nx = x[:, indices]

In [149]:
# evaluate_corr(x, y)
y_oh = np.array([p2oh(pred) for pred in y.squeeze()])
corrs = np.zeros((x.shape[1], y_oh.shape[1]))
for var_idx in range(x.shape[1]):
    for y_idx in range(y_oh.shape[1]):
        corr = evaluate_corr(x[:, var_idx], y_oh[:, y_idx])[0]
        corrs[var_idx, y_idx] = corr
        print('{} - {}: {}'.format(var_idx, y_idx, corr))

0 - 0: -0.17093582817624034
0 - 1: 0.17093582817624034


In [150]:
np.mean(np.abs(corrs))

0.17093582817624034

In [151]:
np.max(np.abs(corrs))

0.17093582817624034

In [140]:
np.min(corrs)

-0.1054613934799101

In [141]:
from collections import Counter
m = Counter(y)
m

Counter({0: 612, 2: 1279, 1: 3252, 3: 912, 4: 71})

In [142]:
print('Uninformed guess acc')
max(m.values()) / sum(m.values())


Uninformed guess acc


0.5308521057786484

In [126]:

def get_sorted_idx(corrs):
    max_corrs = np.max(np.abs(corrs), axis=1)
    arr = [(val, idx) for idx, val in enumerate(max_corrs)]
    arr.sort(reverse=True)
    return arr
out = get_sorted_idx(corrs)
print(out)
top = out[:20]
indices = [p[1] for p in top]
indices

[(0.17093582817624034, 0)]


[0]

In [143]:
np.mean([t[0] for t in top])

0.17093582817624034